In [ ]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions')
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-llama2-7b-32k-instructions', 
    use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="cuda:0"
)

[2023-11-10 07:59:49,015] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from tqdm import tqdm
import os
import random

In [4]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [5]:
arange = set(range(len(questions)))

In [6]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [7]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [8]:
i = 1
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Kebanyakan barang yang disimpan di dalam stor itu telah rosak ____ tikus.
A. digerit
B. digigit
C. dikesip
D. diketip
jawapan:


In [9]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': prompt}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,  1450,
           557,   594,   284,   801,   319, 29902,   343,   574,   286,  1160,
         29884,   432,  1450,   370,  2377,  2883,   577,   284,   273,    13,
         29966,   829, 14816, 29903,  6778,    13,    13,   711,  9761,   361,
         29901,   365,   996, 21474, 11052, 10156,   271, 29899,   388,   271,
           343,   574,  7655,   638,   329,   972,  6249,  2626,  2638, 29882,
           432,  1450, 21419,   343,   574,  5112,   292,  3999, 29884,  1794,
         29889,    13,   578,   284,   273, 29901,   476,   774,  1384,   557,
           273,  2594,   574,   343,   574,   766,  6574,   273,   652,  2959,
           314,  2840,   372, 29884, 13547,   801, 14652,   557,   903, 22359,
           260, 17342, 29889,    13, 29909, 29889,   270,  4087,   277,    13,
         29933, 29889,  4697,   335,   277,    13, 29907, 29889,   270, 29379,
           666,    13, 29928, 29889,  

In [14]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=100,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
# print(tokenizer.decode(r[0]).split('jawapan:')[1].strip().split())

In [15]:
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nobjektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.\nsoalan: Kebanyakan barang yang disimpan di dalam stor itu telah rosak ____ tikus.\nA. digerit\nB. digigit\nC. dikesip\nD. diketip\njawapan: [/INST] Jawapannya ialah C, dikesip. </s>'

In [17]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_llama_chat(messages)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=100,
                top_p=0.95,
                top_k=50,
                temperature=0.3,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

100%|██████████| 54/54 [01:40<00:00,  1.85s/it]


In [21]:
import json

with open('output-0shot-llama2-7b-32k-instructions.json', 'w') as fopen:
    json.dump(questions, fopen)

In [22]:
def most_common(l):
    return max(set(l), key=l.count)

In [23]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

35.294117647058826